In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
from pathlib import Path
import pandas as pd

import numpy as np

from helpers import *

from joblib import dump, load
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier)
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm # svm.SVC(kernel='linear'), svm.SVC(kernel='rbf')
from sklearn.ensemble import VotingRegressor

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [8]:
# --------------------------------------------------------------------------

# global vars

# --------------------------------------------------------------------------

PROJ = Path(r".")
# put data here
DATA = PROJ/'data' 
RAW = DATA/'raw'
data_files = list(RAW.iterdir())
[str(f) for f in data_files]

['data/raw/.ipynb_checkpoints',
 'data/raw/data description.csv',
 'data/raw/example_code.py',
 'data/raw/testing.csv',
 'data/raw/training.csv']

In [9]:
# desc = pd.read_csv(data_files[0])
# display_all(desc)
test_data = pd.read_csv('data/raw/testing.csv', index_col=0)
display_all(test_data.head())

,Area_2015,Area_2013,Population_Density_2018,Population _2018,Population_Density_2017,Population _2017,Population_Density_2016,Population _2016,Population_Density_2015,Population _2015,Population_Density_2013,Population _2013,Population_Density_2010,Population_2010,Population_Density_2000,Population _2000,Age_of_Residents_2017,Age_of_Residents_moe_2017,Youth_Population_2017,Youth_Population_moe_2017,Older_Adult_Population_2017,Older_Adult_Population_moe_2017,Vacant_Land_2018,Vacant_Land_Area_2018,Vacant_Land_2017,Vacant_Land_Area_2017,Vacant_Land_2016,Vacant_Land_Area_2016,Vacant_Land_2015,Vacant_Land_Area_2015,Vacant_Land_2013,Vacant_Land_Area_2013,Vacant_Land_2011,Vacant_Land_Area_2011,Household_Income_2017,Household_Income_moe_2017,Public_Nutrition_Assistance_2018,Public_Nutrition_Assistance_2017,Public_Nutrition_Assistance_2016,Public_Nutrition_Assistance_2015,Public_Nutrition_Assistance_2013,Public_Nutrition_Assistance_2011,Employment_Rate_2017,Employment_Rate_moe_2017,Commercial_Construction_2018,Commercial_Construction_Permitted_Units_2018,Commercial_Construction_2017,Commercial_Construction_Permitted_Units_2017,Commercial_Construction_2016,Commercial_Construction_Permitted_Units_2016,Commercial_Construction_2015,Commercial_Construction_Permitted_Units_2015,Commercial_Construction_2014,Commercial_Construction_Permitted_Units_2014,Commercial_Construction_2013,Commercial_Construction_Permitted_Units_2013,Commercial_Construction_2012,Commercial_Construction_Permitted_Units_2012,Commercial_Construction_2011,Commercial_Construction_Permitted_Units_2011,Commercial_Size_2018,Commercial_Size_Total_2018,Commercial_Size_2017,Commercial_Size_Total_2017,Commercial_Size_2016,Commercial_Size_Total_2016,Commercial_Size_2015,Commercial_Size_Total_2015,Commercial_Size_2013,Commercial_Size_Total_2013,Commercial_Size_2011,Commercial_Size_Total_2011,Commercial_Building_Age_2018,Commercial_Building_Age_2017,Commercial_Building_Age_2016,Commercial_Building_Age_2015,Commercial_Building_Age_2013,Commercial_Building_Age_11,Fincancial_Services_Proximity_2018,Financial_Services_Proximate_Units_2018,Fincancial_Services_Proximity_2017,Financial_Services_Proximate_Units_2017,Fincancial_Services_Proximity_2016,Financial_Services_Proximate_Units_2016,Fincancial_Services_Proximity_2015,Financial_Services_Proximate_Units_2015,Fincancial_Services_Proximity_2014,Financial_Services_Proximate_Units_2014,Bachelors_Degree_2017,Bachelors_Degree_moe_2017,High_School_Diploma_2017,High_School_Diploma_moe_2017,Early_Care_Proximity_2017,Early_Care_Proximate_Units_2017,Early_Care_Proximity_2015,Early_Care_Proximate_Units_2015,Early_Care_Proximity_2013,Early_Care_Proximate_Units_2013,Early_Care_Proximity_2011,Early_Care_Proximate_Units_2011,SchoolAge_Proximity_2017,SchoolAge_Proximate_Units_2017,SchoolAge_Proximity_2015,SchoolAge_Proximate_Units_2015,SchoolAge_Proximity_2013,SchoolAge_Proximate_Units_2013,SchoolAge_Proximity_2011,SchoolAge_Proximate_Units_2011,Library_Card_Prevalence_2015,Library_Card_Holders_2015,Library_Card_Prevalence_2013,Library_Card_Holders_2013,Proficiency_Elementary_School_2017,Proficiency_Elementary_School_2016,Proficiency_Elementary_School_2015,Proficiency_Elementary_School_2014,Proficiency_Middle_School_2017,Proficiency_Middle_School_2016,Proficiency_Middle_School_2015,Proficiency_Middle_School_2014,Proficiency_High_School_2017,Proficiency_High_School_2016,Proficiency_High_School_2015,Proficiency_High_School_2014,Highschool_Graduation_Rate_2017,Highschool_Graduation_Rate_2016,Highschool_Graduation_Rate_2015,Highschool_Graduation_Rate_2014,Highschool_Graduation_Rate_2013,Student_Absenteeism_2017,Student_Absenteeism_2016,Student_Absenteeism_2015,Student_Absenteeism_2014,Student_Absenteeism_2013,Neighborhood_School_Attendance_2017,Neighborhood_School_Attendance_2016,Neighborhood_School_Attendance_2015,Neighborhood_School_Attendance_2014,Neighborhood_School_Attendance_2013,Neighborhood_School_Attendance_2011,Arts_Participation_2013,Arts_Partic

## data

In [10]:
df = pd.read_csv('data/raw/training.csv', low_memory=False, index_col=0)#
display_all(df.info(3))
display_all(df.head(3).T)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416 entries, 0 to 415
Data columns (total 496 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   Area_2015                                     int64  
 1   Area_2013                                     int64  
 2   Population_Density_2018                       int64  
 3   Population _2018                              int64  
 4   Population_Density_2017                       int64  
 5   Population _2017                              int64  
 6   Population_Density_2016                       int64  
 7   Population _2016                              int64  
 8   Population_Density_2015                       int64  
 9   Population _2015                              int64  
 10  Population_Density_2013                       int64  
 11  Population _2013                              int64  
 12  Population_Density_2010                       int64  
 13  Popu

None

,0,1,2
Area_2015,4.110000e+02,1.156000e+03,329.000000
Area_2013,4.110000e+02,1.156000e+03,329.000000
Population_Density_2018,5.000000e+00,8.000000e+00,3.000000
Population _2018,2.243000e+03,9.758000e+03,1120.000000
Population_Density_2017,5.000000e+00,8.000000e+00,3.000000
Population _2017,2.241000e+03,8.816000e+03,1089.000000
Population_Density_2016,5.000000e+00,7.000000e+00,3.000000
Population _2016,2.240000e+03,7.986000e+03,1080.000000
Population_Density_2015,5.000000e+00,7.000000e+00,3.000000
Population _2015,2.216000e+03,7.938000e+03,1078.000000


### stats and nulls?

In [11]:
display_all(df.describe())
display_all(df.isnull().sum())

,Area_2015,Area_2013,Population_Density_2018,Population _2018,Population_Density_2017,Population _2017,Population_Density_2016,Population _2016,Population_Density_2015,Population _2015,Population_Density_2013,Population _2013,Population_Density_2010,Population_2010,Population_Density_2000,Population _2000,Age_of_Residents_2017,Age_of_Residents_moe_2017,Youth_Population_2017,Youth_Population_moe_2017,Older_Adult_Population_2017,Older_Adult_Population_moe_2017,Vacant_Land_2018,Vacant_Land_Area_2018,Vacant_Land_2017,Vacant_Land_Area_2017,Vacant_Land_2016,Vacant_Land_Area_2016,Vacant_Land_2015,Vacant_Land_Area_2015,Vacant_Land_2013,Vacant_Land_Area_2013,Vacant_Land_2011,Vacant_Land_Area_2011,Household_Income_2017,Household_Income_moe_2017,Public_Nutrition_Assistance_2018,Public_Nutrition_Assistance_2017,Public_Nutrition_Assistance_2016,Public_Nutrition_Assistance_2015,Public_Nutrition_Assistance_2013,Public_Nutrition_Assistance_2011,Employment_Rate_2017,Employment_Rate_moe_2017,Commercial_Construction_2018,Commercial_Construction_Permitted_Units_2018,Commercial_Construction_2017,Commercial_Construction_Permitted_Units_2017,Commercial_Construction_2016,Commercial_Construction_Permitted_Units_2016,Commercial_Construction_2015,Commercial_Construction_Permitted_Units_2015,Commercial_Construction_2014,Commercial_Construction_Permitted_Units_2014,Commercial_Construction_2013,Commercial_Construction_Permitted_Units_2013,Commercial_Construction_2012,Commercial_Construction_Permitted_Units_2012,Commercial_Construction_2011,Commercial_Construction_Permitted_Units_2011,Commercial_Size_2018,Commercial_Size_Total_2018,Commercial_Size_2017,Commercial_Size_Total_2017,Commercial_Size_2016,Commercial_Size_Total_2016,Commercial_Size_2015,Commercial_Size_Total_2015,Commercial_Size_2013,Commercial_Size_Total_2013,Commercial_Size_2011,Commercial_Size_Total_2011,Commercial_Building_Age_2018,Commercial_Building_Age_2017,Commercial_Building_Age_2016,Commercial_Building_Age_2015,Commercial_Building_Age_2013,Commercial_Building_Age_11,Fincancial_Services_Proximity_2018,Financial_Services_Proximate_Units_2018,Fincancial_Services_Proximity_2017,Financial_Services_Proximate_Units_2017,Fincancial_Services_Proximity_2016,Financial_Services_Proximate_Units_2016,Fincancial_Services_Proximity_2015,Financial_Services_Proximate_Units_2015,Fincancial_Services_Proximity_2014,Financial_Services_Proximate_Units_2014,Bachelors_Degree_2017,Bachelors_Degree_moe_2017,High_School_Diploma_2017,High_School_Diploma_moe_2017,Early_Care_Proximity_2017,Early_Care_Proximate_Units_2017,Early_Care_Proximity_2015,Early_Care_Proximate_Units_2015,Early_Care_Proximity_2013,Early_Care_Proximate_Units_2013,Early_Care_Proximity_2011,Early_Care_Proximate_Units_2011,SchoolAge_Proximity_2017,SchoolAge_Proximate_Units_2017,SchoolAge_Proximity_2015,SchoolAge_Proximate_Units_2015,SchoolAge_Proximity_2013,SchoolAge_Proximate_Units_2013,SchoolAge_Proximity_2011,SchoolAge_Proximate_Units_2011,Library_Card_Prevalence_2015,Library_Card_Holders_2015,Library_Card_Prevalence_2013,Library_Card_Holders_2013,Proficiency_Elementary_School_2017,Proficiency_Elementary_School_2016,Proficiency_Elementary_School_2015,Proficiency_Elementary_School_2014,Proficiency_Middle_School_2017,Proficiency_Middle_School_2016,Proficiency_Middle_School_2015,Proficiency_Middle_School_2014,Proficiency_High_School_2017,Proficiency_High_School_2016,Proficiency_High_School_2015,Proficiency_High_School_2014,Highschool_Graduation_Rate_2017,Highschool_Graduation_Rate_2016,Highschool_Graduation_Rate_2015,Highschool_Graduation_Rate_2014,Highschool_Graduation_Rate_2013,Student_Absenteeism_2017,Student_Absenteeism_2016,Student_Absenteeism_2015,Student_Absenteeism_2014,Student_Absenteeism_2013,Neighborhood_School_Attendance_2017,Neighborhood_School_Attendance_2016,Neighborhood_School_Attendance_2015,Neighborhood_School_Attendance_2014,Neighborhood_School_Attendance_2013,Neighborhood_School_Attendance_2011,Arts_Participation_2013,Arts_Partic

Area_2015                                         0
Area_2013                                         0
Population_Density_2018                           0
Population _2018                                  0
Population_Density_2017                           0
Population _2017                                  0
Population_Density_2016                           0
Population _2016                                  0
Population_Density_2015                           0
Population _2015                                  0
Population_Density_2013                           0
Population _2013                                  0
Population_Density_2010                           0
Population_2010                                   0
Population_Density_2000                           0
Population _2000                                  0
Age_of_Residents_2017                             1
Age_of_Residents_moe_2017                        57
Youth_Population_2017                             1
Youth_Popula

## simplest naive model 
The simplest non-stupid model we came up with last week was create a column of the mean and submit that to Kaggle.

In [33]:
X = df.drop('target', axis=1).copy()
y = df['target'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [34]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((312, 495), (104, 495), (312,), (104,))

In [25]:
# clean the data
lr_pipe = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')), 
                 ('scaler', StandardScaler()), 
                 ('lr', LinearRegression())])

lr_pipe.fit(X_train, y_train)
print("Test score: {:.2f}".format(lr_pipe.score(X_test, y_test )))

Test score: -1.09


In [23]:
# clean the data
knn_pipe = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')), 
                 ('scaler', StandardScaler()), 
                 ('knn', KNeighborsRegressor(n_neighbors=6))])

knn_pipe.fit(X_train, y_train)
print("Test score: {:.2f}".format(knn_pipe.score(X_test, y_test )))

Test score: -0.02


In [18]:
# clean the data
pipe = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')), 
                 ('scaler', StandardScaler()), 
                 ('rforest', RandomForestRegressor(random_state=42))])

pipe.fit(X_train, y_train)
print("Test score: {:.2f}".format(pipe.score(X_test, y_test )))

Test score: -0.02


In [40]:
metrics.mean_squared_error(y_test,pipe.predict(X_test))

1.6063613666797492

In [17]:
dump(pipe, 'models/rf_regression_1.joblib')

['models/rf_regression_1.joblib']

In [36]:
trans = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
                      ('scaler', StandardScaler())])
trans.fit(X_train)
trans_train = trans.transform(X_train)
trans_data = trans.transform(X_test)
trans_data.shape
vote3 = VotingRegressor([('lr',LinearRegression()), 
                         ('knn', KNeighborsRegressor(n_neighbors=6)), 
                         ('rf', RandomForestRegressor(random_state=42))])
vote3.fit(trans_train, y_train)

VotingRegressor(estimators=[('lr',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('knn',
                             KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                                 metric='minkowski',
                                                 metric_params=None,
                                                 n_jobs=None, n_neighbors=6,
                                                 p=2, weights='uniform')),
                            ('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                          

In [39]:
preds_vote3 = vote3.predict(trans_data)
metrics.mean_squared_error(y_test,preds_vote3)

1.9887815397148076

In [40]:
def save_prediction_to_csv(y_pred):
    """
    Use this function to save your prediction result to a csv file.
    The resulting csv file is named as [team_name].csv

    :param y_pred: an array or a pandas series that follows the SAME index order as in the testing data
    """
    pd.DataFrame(dict(
        target=y_pred
    )).to_csv('predictions.csv', index=False, header=False)

In [38]:
y_pred = pipe.predict(test_data)

In [41]:
save_prediction_to_csv(y_pred)